In [4]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re


In [5]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Notebooks/Data/credit_train.csv', encoding='cp1251', sep=';')
# смотрим форму датафрейма
print(data.shape)

# выводим первые 5 наблюдений датафрейма
data.head()

(170746, 15)


,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


In [6]:
# удаляем переменную client_id.
# Эта переменная имеет столько же уникальных значений, сколько у нас наблюдений, поэтому бес-
# полезна для моделирования
data.drop('client_id', axis='columns', inplace=True)
# выводим первые 5 наблюдений датафрейма
data.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


Обратите внимание на странное значение ОБЛ САРАТОВСКАЯ переменной living_region.
Давайте с помощью метода .unique() выведем уникальные значения этой переменной.

In [7]:
# смотрим уникальные значения по переменной living_region
data['living_region'].unique()

array(['КРАСНОДАРСКИЙ КРАЙ', 'МОСКВА', 'ОБЛ САРАТОВСКАЯ',
       'ОБЛ ВОЛГОГРАДСКАЯ', 'ЧЕЛЯБИНСКАЯ ОБЛАСТЬ', 'СТАВРОПОЛЬСКИЙ КРАЙ',
       'ОБЛ НИЖЕГОРОДСКАЯ', 'МОСКОВСКАЯ ОБЛ',
       'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА', 'КРАЙ СТАВРОПОЛЬСКИЙ',
       'САНКТ-ПЕТЕРБУРГ', 'РЕСП. БАШКОРТОСТАН', 'ОБЛ АРХАНГЕЛЬСКАЯ',
       'ХАНТЫ-МАНСИЙСКИЙ АО', 'РЕСП БАШКОРТОСТАН', 'ПЕРМСКИЙ КРАЙ',
       'РЕСП КАРАЧАЕВО-ЧЕРКЕССКАЯ', 'САРАТОВСКАЯ ОБЛ', 'ОБЛ КАЛУЖСКАЯ',
       'ОБЛ ВОЛОГОДСКАЯ', 'РОСТОВСКАЯ ОБЛ', 'УДМУРТСКАЯ РЕСП',
       'ОБЛ ИРКУТСКАЯ', 'ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ', 'ОБЛ МОСКОВСКАЯ',
       'ОБЛ ТЮМЕНСКАЯ', 'ОБЛ БЕЛГОРОДСКАЯ', 'РОСТОВСКАЯ ОБЛАСТЬ',
       'ОБЛ КОСТРОМСКАЯ', 'РЕСП ХАКАСИЯ', 'РЕСПУБЛИКА ТАТАРСТАН',
       'ИРКУТСКАЯ ОБЛАСТЬ', 'ОБЛ СВЕРДЛОВСКАЯ', 'ОБЛ ПСКОВСКАЯ',
       'КРАЙ ЗАБАЙКАЛЬСКИЙ', 'СВЕРДЛОВСКАЯ ОБЛ', 'ОБЛ ОРЕНБУРГСКАЯ',
       'ОБЛ ВОРОНЕЖСКАЯ', 'ОБЛ АСТРАХАНСКАЯ', 'ОБЛ НОВОСИБИРСКАЯ',
       'ОБЛ ЧЕЛЯБИНСКАЯ', 'ОРЕНБУРГСКАЯ ОБЛ', 'СВЕРДЛОВСКАЯ ОБЛАСТЬ'

In [8]:
# увеличиваем максимальное количество отображаемых строк
pd.options.display.max_rows = 310

# уникальные значения переменной living_region записываем в отдельный объект regions
regions = data['living_region'].unique()

In [9]:
# создаем серию, у которой в качестве значений и индексных меток будут выступать
# уникальные значения переменной living_region, записанные в regions
regions = pd.Series(data=regions, index=regions, name='regions')
regions.head()

КРАСНОДАРСКИЙ КРАЙ      КРАСНОДАРСКИЙ КРАЙ
МОСКВА                              МОСКВА
ОБЛ САРАТОВСКАЯ            ОБЛ САРАТОВСКАЯ
ОБЛ ВОЛГОГРАДСКАЯ        ОБЛ ВОЛГОГРАДСКАЯ
ЧЕЛЯБИНСКАЯ ОБЛАСТЬ    ЧЕЛЯБИНСКАЯ ОБЛАСТЬ
Name: regions, dtype: object

In [10]:
# задаем стоп-слова
stopwrds = set(['ОБЛ', 'ОБЛАСТЬ', 'РЕСП', 'РЕСПУБЛИКА',
'КРАЙ', 'Г', 'АО', 'АОБЛ', 'АВТОНОМНАЯ'])

# пишем функцию для предобработки значений серии
def clean_region(x):
    x = re.sub('[.,]+', ' ', str(x))
    wrds = x.split(' ')
    wrds_new = []
    for w in wrds:
        if not w in stopwrds:
            wrds_new.append(w)
    x = ''.join(wrds_new)
    return x

# применяем функцию к нашей серии
regions = regions.map(clean_region)

# смотрим результаты применения функции
regions

КРАСНОДАРСКИЙ КРАЙ                                                 КРАСНОДАРСКИЙ
МОСКВА                                                                    МОСКВА
ОБЛ САРАТОВСКАЯ                                                      САРАТОВСКАЯ
ОБЛ ВОЛГОГРАДСКАЯ                                                  ВОЛГОГРАДСКАЯ
ЧЕЛЯБИНСКАЯ ОБЛАСТЬ                                                  ЧЕЛЯБИНСКАЯ
СТАВРОПОЛЬСКИЙ КРАЙ                                               СТАВРОПОЛЬСКИЙ
ОБЛ НИЖЕГОРОДСКАЯ                                                  НИЖЕГОРОДСКАЯ
МОСКОВСКАЯ ОБЛ                                                        МОСКОВСКАЯ
ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА    ХАНТЫ-МАНСИЙСКИЙАВТОНОМНЫЙОКРУГ-ЮГРА
КРАЙ СТАВРОПОЛЬСКИЙ                                               СТАВРОПОЛЬСКИЙ
САНКТ-ПЕТЕРБУРГ                                                  САНКТ-ПЕТЕРБУРГ
РЕСП. БАШКОРТОСТАН                                                  БАШКОРТОСТАН
ОБЛ АРХАНГЕЛЬСКАЯ           

In [11]:
# вносим финальные корректировки в regions
regions['ЧУКОТСКИЙ АO'] = 'ЧУКОТСКИЙ'
regions['ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШИЯ ЧУВАШСКАЯ РЕСПУБЛИКА -'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП'] = 'ЧУВАШСКАЯ'
regions['РЕСП ЧУВАШСКАЯ - ЧУВАШИЯ'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП'] = 'ЧУВАШСКАЯ'
regions['РЕСПУБЛИКАТАТАРСТАН'] = 'ТАТАРСТАН'
regions['ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ'] = 'МОСКОВСКАЯ'
regions['ПЕРМСКАЯ ОБЛ'] = 'ПЕРМСКИЙ'
regions['ОРЁЛ'] = 'ОРЛОВСКАЯ'
regions['Г.ОДИНЦОВО МОСКОВСКАЯ ОБЛ'] = 'МОСКОВСКАЯ'
regions['МЫТИЩИНСКИЙ Р-Н'] = 'МОСКОВСКАЯ'
regions['МОСКОВСКИЙ П'] = 'МОСКОВСКАЯ'
regions['КАМЧАТСКАЯ ОБЛАСТЬ'] = 'КАМЧАТСКИЙ'
regions['ДАЛЬНИЙ ВОСТОК'] = 'МОСКОВСКАЯ'
regions['ДАЛЬНИЙВОСТОК'] = 'МОСКОВСКАЯ'
regions['ГУСЬ-ХРУСТАЛЬНЫЙ Р-Н'] = 'ВЛАДИМИРСКАЯ'
regions['ГОРЬКОВСКАЯ ОБЛ'] = 'НИЖЕГОРОДСКАЯ'
regions['ЭВЕНКИЙСКИЙ АО'] = 'КРАСНОЯРСКИЙ'
regions['ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions['АО ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - Ю'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions['АО ХАНТЫ-МАНСИЙСКИЙ-ЮГРА'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions['СЕВ. ОСЕТИЯ - АЛАНИЯ'] = 'СЕВЕРНАЯОСЕТИЯ-АЛАНИЯ'
regions['РЕСП. САХА (ЯКУТИЯ)'] = 'САХА/ЯКУТИЯ/'
regions['РЕСПУБЛИКА САХА'] = 'САХА/ЯКУТИЯ/'
regions['ДАЛЬНИЙВОСТОК'] = 'МОСКОВСКАЯ'
regions['САХА'] = 'САХА/ЯКУТИЯ/'
regions['98'] = 'САНКТ-ПЕТЕРБУРГ'
regions['74'] = 'ЧЕЛЯБИНСКАЯ'
regions['РОССИЯ'] = 'МОСКОВСКАЯ'
regions['МОСКВОСКАЯ'] = 'МОСКОВСКАЯ'
regions['МОСКВОСКАЯ ОБЛ'] = 'МОСКОВСКАЯ'
regions['ЧЕЛЯБИНСК'] = 'ЧЕЛЯБИНСКАЯ'
regions['Г. ЧЕЛЯБИНСК'] = 'ЧЕЛЯБИНСКАЯ'
regions['БРЯНСКИЙ'] = 'БРЯНСКАЯ'

In [12]:
# вновь смотрим серию
regions

КРАСНОДАРСКИЙ КРАЙ                                  КРАСНОДАРСКИЙ
МОСКВА                                                     МОСКВА
ОБЛ САРАТОВСКАЯ                                       САРАТОВСКАЯ
ОБЛ ВОЛГОГРАДСКАЯ                                   ВОЛГОГРАДСКАЯ
ЧЕЛЯБИНСКАЯ ОБЛАСТЬ                                   ЧЕЛЯБИНСКАЯ
СТАВРОПОЛЬСКИЙ КРАЙ                                СТАВРОПОЛЬСКИЙ
ОБЛ НИЖЕГОРОДСКАЯ                                   НИЖЕГОРОДСКАЯ
МОСКОВСКАЯ ОБЛ                                         МОСКОВСКАЯ
ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА         ХАНТЫ-МАНСИЙСКИЙ
КРАЙ СТАВРОПОЛЬСКИЙ                                СТАВРОПОЛЬСКИЙ
САНКТ-ПЕТЕРБУРГ                                   САНКТ-ПЕТЕРБУРГ
РЕСП. БАШКОРТОСТАН                                   БАШКОРТОСТАН
ОБЛ АРХАНГЕЛЬСКАЯ                                   АРХАНГЕЛЬСКАЯ
ХАНТЫ-МАНСИЙСКИЙ АО                              ХАНТЫ-МАНСИЙСКИЙ
РЕСП БАШКОРТОСТАН                                    БАШКОРТОСТАН
ПЕРМСКИЙ К

In [13]:
# заменяем исходные категории переменной living_region на новые
data['living_region'] = data['living_region'].map(regions)

# смотрим уникальные значения по переменной living_region
data['living_region'].unique()

array(['КРАСНОДАРСКИЙ', 'МОСКВА', 'САРАТОВСКАЯ', 'ВОЛГОГРАДСКАЯ',
       'ЧЕЛЯБИНСКАЯ', 'СТАВРОПОЛЬСКИЙ', 'НИЖЕГОРОДСКАЯ', 'МОСКОВСКАЯ',
       'ХАНТЫ-МАНСИЙСКИЙ', 'САНКТ-ПЕТЕРБУРГ', 'БАШКОРТОСТАН',
       'АРХАНГЕЛЬСКАЯ', 'ПЕРМСКИЙ', 'КАРАЧАЕВО-ЧЕРКЕССКАЯ', 'КАЛУЖСКАЯ',
       'ВОЛОГОДСКАЯ', 'РОСТОВСКАЯ', 'УДМУРТСКАЯ', 'ИРКУТСКАЯ',
       'ТЮМЕНСКАЯ', 'БЕЛГОРОДСКАЯ', 'КОСТРОМСКАЯ', 'ХАКАСИЯ', 'ТАТАРСТАН',
       'СВЕРДЛОВСКАЯ', 'ПСКОВСКАЯ', 'ЗАБАЙКАЛЬСКИЙ', 'ОРЕНБУРГСКАЯ',
       'ВОРОНЕЖСКАЯ', 'АСТРАХАНСКАЯ', 'НОВОСИБИРСКАЯ', 'КУРГАНСКАЯ',
       'УЛЬЯНОВСКАЯ', 'МУРМАНСКАЯ', 'КРАСНОЯРСКИЙ', 'БУРЯТИЯ',
       'САХА/ЯКУТИЯ/', 'АМУРСКАЯ', 'ХАБАРОВСКИЙ', 'ЯМАЛО-НЕНЕЦКИЙ',
       'САМАРСКАЯ', 'ТВЕРСКАЯ', 'ЯРОСЛАВСКАЯ', 'ВЛАДИМИРСКАЯ',
       'ЛЕНИНГРАДСКАЯ', 'ОРЛОВСКАЯ', 'КЕМЕРОВСКАЯ', 'ОМСКАЯ', 'ЧЕЧЕНСКАЯ',
       'КУРСКАЯ', 'ТУЛЬСКАЯ', 'АДЫГЕЯ', 'КОМИ', 'ПРИМОРСКИЙ',
       'СМОЛЕНСКАЯ', 'КИРОВСКАЯ', 'ДАГЕСТАН', 'ПЕНЗЕНСКАЯ', 'КАРЕЛИЯ',
       'ТОМСКАЯ', 'МАГАДАНСКАЯ', 'МАРИЙЭЛ', 'ИВАНО

In [14]:
# смотрим количество уникальных значений переменной living_region
data['living_region'].nunique()

85

In [15]:
# смотрим типы переменных
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170744 non-null  object 
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  float64
 7   score_shk             170739 non-null  object 
 8   education             170741 non-null  object 
 9   living_region         170746 non-null  object 
 10  monthly_income        170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  open_account_flg      170746 non-null  int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 18.

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0


Видно, что категориальные переменные tariff_id и open_account_flg неверно записаны как количественные переменные: первой присвоен тип float (используется для представления чисел с плавающей точкой), а второй – тип int (используется для представления целых чисел).

In [16]:
# преобразуем указанные переменные в тип object
for i in ['tariff_id', 'open_account_flg']:
    data[i] = data[i].astype('object')

Количественные переменные credit_sum и score_shk, наоборот, неверно записаны как категориальные переменные, им присвоен тип object. Это обусловлено тем, что вместо точки в качестве десятичного разделителя использовалась запятая. Кроме того, мы видим, что многие переменные имеют пропуски.

In [17]:
# в указанных переменных заменяем запятые на точки и преобразуем в тип float
for i in ['credit_sum', 'score_shk']:
    data[i] = data[i].str.replace(',', '.').astype('float')

# смотрим типы переменных
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170744 non-null  float64
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  object 
 7   score_shk             170739 non-null  float64
 8   education             170741 non-null  object 
 9   living_region         170746 non-null  object 
 10  monthly_income        170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  open_account_flg      170746 non-null  object 
dtypes: float64(6), int64(1), object(7)
memory usage: 18.

#### Переименование категорий переменных
Иногда бывают ситуации, когда нужно переменовать категории переменной. Для
этого можно использовать метод map, передав ему в качестве аргумента словарь
вида {старое название категории: новое название категории}.

In [18]:
# создаем словарь, в котором ключом является старое
# название категории, значением – новое название категории
d = {'M':'Male', 'F':'Female'}

# передаем словарь в метод map
data['gender'] = data['gender'].map(d)
data.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,Male,NaN,NaN,UMN,59998.00,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,Female,NaN,MAR,UMN,10889.00,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,Male,32.0,MAR,SPC,10728.00,12,1.1,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,Female,27.0,NaN,SPC,12009.09,12,1.1,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,Male,45.0,NaN,SPC,NaN,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0


Аналогичную операцию можно выполнить с помощью метода replace()

In [19]:
# создаем словарь, в котором ключом является старое
# название категории, значением – новое название категории
f = {'Male':'M', 'Female':'F'}

# передаем в метод replace словарь, в котором ключом будет название переменной,
# а значением – словарь со старыми и новыми названиями категорий
data = data.replace({'gender': f})
data.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,59998.00,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,10889.00,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,10728.00,12,1.1,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,12009.09,12,1.1,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0


#### Обработка редких категорий
Часто бывает, что наши переменные содержат редкие категории. Редкие категории
являются источником шума в данных, который негативно повлияет на качество
модели. Кроме того, при разбиении набора данных на обучающую и контрольную
выборки может оказаться, что данная категория отсутствует в обучающей выборке,
но присутствует в контрольной выборке. Это вызовет проблемы при моделировании.
Например, регрессионная модель, встретив в новых данных наблюдение
с неизвестной категорией предиктора, не сможет вычислить прогноз, потому что
необходимый для прогноза регрессионный коэффициент по этой категории предиктора будет отсутствовать.

In [20]:
# выводим частоты категорий по каждой категориальной переменной
# categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
# альтернативный вариант строки выше
categorical_columns_2 = data.select_dtypes(include='object').columns
for c in categorical_columns_2:
    print(data[c].value_counts())

gender
F    88697
M    82049
Name: count, dtype: int64
marital_status
MAR    93954
UNM    52149
DIV    16969
CIV     4196
WID     3475
Name: count, dtype: int64
job_position
SPC    134680
UMN     17674
BIS      5591
PNA      4107
DIR      3750
ATP      2791
WRK       656
NOR       537
WOI       352
INP       241
BIU       126
WRP       110
PNI        65
PNV        40
PNS        12
HSK         8
INV         5
ONB         1
Name: count, dtype: int64
tariff_id
1.1     69355
1.6     39117
1.32    15537
1.4     10970
1.5      7497
1.9      5538
1.43     3930
1.3      3339
1.16     3232
1.0      2245
1.44     2228
1.19     2102
1.2      1306
1.7      1007
1.17      717
1.21      579
1.94      414
1.22      376
1.23      370
1.91      317
1.24      303
1.41      132
1.25       56
1.18       36
1.26       11
1.28       10
1.52        7
1.27        6
1.48        5
1.56        2
1.96        1
1.29        1
Name: count, dtype: int64
education
SCH    87537
GRD    72591
UGR     9941
PGR      565
AC

**Внимательный анализ показывает, что переменные job_position, tariff_id и living_region содержат множественные редкие категории.**

Выведем частоты категорий по переменной job_position с помощью метода .value_counts(). Обратите внимание, по умолчанию пропуски не выводятся, и чтобы их вывести, необходимо для параметра dropna метода .value_counts() задать значение False.

In [21]:
# выводим частоты категорий для переменной job_position,
# dropna=False выведет частоту пропусков, если они есть
job_pos_val_counts = data['job_position'].value_counts(dropna=False)
print(job_pos_val_counts)

job_position
SPC    134680
UMN     17674
BIS      5591
PNA      4107
DIR      3750
ATP      2791
WRK       656
NOR       537
WOI       352
INP       241
BIU       126
WRP       110
PNI        65
PNV        40
PNS        12
HSK         8
INV         5
ONB         1
Name: count, dtype: int64


Видно, что переменная job_position не содержит пропусков. Все категории переменной
job_position с частотой менее 55 наблюдений (это категории PNV, PNS, HSK, INV
и ONB) мы объединим в отдельную категорию OTHER.

In [22]:
# записываем указанные категории переменной job_position в отдельную категорию OTHER
low = data['job_position'].value_counts(dropna=False) < 50
low_index = job_pos_val_counts[low].index
# создаем словарь для замены
dict_other = {a: 'OTHER' for a in low_index.values}
data.replace({'job_position': dict_other}, inplace=True)

In [23]:
# выводим частоты категорий для переменной job_position
print(data['job_position'].value_counts(dropna=False))

job_position
SPC      134680
UMN       17674
BIS        5591
PNA        4107
DIR        3750
ATP        2791
WRK         656
NOR         537
WOI         352
INP         241
BIU         126
WRP         110
OTHER        66
PNI          65
Name: count, dtype: int64


In [24]:
# выводим частоты категорий для переменной tariff_id
print(data['tariff_id'].value_counts(dropna=False))

tariff_id
1.1     69355
1.6     39117
1.32    15537
1.4     10970
1.5      7497
1.9      5538
1.43     3930
1.3      3339
1.16     3232
1.0      2245
1.44     2228
1.19     2102
1.2      1306
1.7      1007
1.17      717
1.21      579
1.94      414
1.22      376
1.23      370
1.91      317
1.24      303
1.41      132
1.25       56
1.18       36
1.26       11
1.28       10
1.52        7
1.27        6
1.48        5
1.56        2
1.96        1
1.29        1
Name: count, dtype: int64


In [25]:
# все категории переменной tariff_id с частотой
# менее 55 наблюдений записываем в отдельную категорию 1.99
# ОТЛИЧНЫЙ ПРИЕМ как всем значениям проставить их счетчик (не смотря на дубли)
# data['tariff_id'].value_counts()[data['tariff_id']]
data.loc[data['tariff_id'].value_counts()[data['tariff_id']].values < 55, 'tariff_id'] = 1.99

# на основе категориальной переменной tariff_id создаем
# количественную переменную tariff
data['tariff'] = data['tariff_id'].astype('float')

# заменим точки на символы подчеркивания
data['tariff_id'] = data['tariff_id'].astype('str').str.replace('.', '_')

Обратите внимание, что этот программный код, выполняющий укрупнение
категорий, сработает только в том случае, когда у вас отсутствуют значения NaN,
в противном случае будет выдана ошибка, поскольку нельзя выполнить индексацию
с помощью вектора, содержащего значения NA/NaN (cannot index with vector
containing NA/NaN values).

In [26]:
# выводим частоты категорий для переменной tariff_id
print(data['tariff_id'].value_counts(dropna=True))

tariff_id
1_1     69355
1_6     39117
1_32    15537
1_4     10970
1_5      7497
1_9      5538
1_43     3930
1_3      3339
1_16     3232
1_0      2245
1_44     2228
1_19     2102
1_2      1306
1_7      1007
1_17      717
1_21      579
1_94      414
1_22      376
1_23      370
1_91      317
1_24      303
1_41      132
1_99       79
1_25       56
Name: count, dtype: int64


Выведем частоты категорий по переменной living_region. Поскольку категорий
очень много, выведем лишь последние 10 категорий.

In [27]:
# выводим частоты для последних 10 категорий переменной living_region
print(data['living_region'].value_counts(dropna=False).tail(10))

living_region
ЕВРЕЙСКАЯ      203
nan            192
НЕНЕЦКИЙ       172
МАГАДАНСКАЯ    159
ДАГЕСТАН        69
АЛТАЙ           54
ЧУКОТСКИЙ       32
ЧЕЧЕНСКАЯ       31
ИНГУШЕТИЯ       19
ЧИТИНСКАЯ       17
Name: count, dtype: int64


In [28]:
# создаем серию, у которой значениями будут частоты категорий переменной living_region
region_series = data['living_region'].value_counts()

# В данном случае мы хотим объединить категории с частотой 50 наблюдений и меньше в ка-
# Обработка редких категорий OTHER. 

# len(data.loc[region_series[data['living_region']].values <= 50])
# data.loc[region_series[data['living_region']].values <= 50, 'living_region'] = 'OTHER'

Альтернативный вариант того что выше

Мы делим 50 наблюдений на общее количество наблюдений в наборе данных (170 746 наблюдений), умножаем на 100 и получаем пороговую относительную частоту 0,029. С помощью программного кода, приведенного ниже, мы делим частоту каждой категории на общее количество наблюдений, умножаем
на 100, получаем относительную частоту и сравниваем ее с пороговым значением
0,029. Если относительная частота категории меньше 0,029, возвращаем значение
TRUE, если больше, то возвращаем значение FALSE.

In [29]:
# создаем булеву маску, если частота категории меньше 0.029,
# будет возвращено значение TRUE, в противном случае
# будет возвращено значение FALSE
mask = (region_series/region_series.sum() * 100).lt(0.029)

# выводим последние 10 категорий
mask.tail(10)

living_region
ЕВРЕЙСКАЯ      False
nan            False
НЕНЕЦКИЙ       False
МАГАДАНСКАЯ    False
ДАГЕСТАН       False
АЛТАЙ          False
ЧУКОТСКИЙ       True
ЧЕЧЕНСКАЯ       True
ИНГУШЕТИЯ       True
ЧИТИНСКАЯ       True
Name: count, dtype: bool

In [30]:
# с помощью функции np.where мы все категории, по которым возвращено
# значение TRUE, заменим на категорию OTHER, в противном случае
# оставим категории неизменными
data['living_region'] = data['living_region']\
                        .mask(data['living_region'].isin(region_series[mask].index), 'OTHER')
# или использовать функционал numpy
# data['living_region'] = np.where(data['living_region'].isin(region_series[mask].index),
#                                 'OTHER', data['living_region'])

In [31]:
# выводим частоты для последних 10 категорий переменной living_region
print(data['living_region'].value_counts(dropna=False).tail(10))

living_region
КАМЧАТСКИЙ               412
СЕВЕРНАЯОСЕТИЯ-АЛАНИЯ    379
КАЛМЫКИЯ                 305
ЕВРЕЙСКАЯ                203
nan                      192
НЕНЕЦКИЙ                 172
МАГАДАНСКАЯ              159
OTHER                     99
ДАГЕСТАН                  69
АЛТАЙ                     54
Name: count, dtype: int64


#### Разбиение набора данных на обучающую и контрольную

In [32]:
# с помощью метода .sample из исходного датафрейма data случайно отбираем
# 70% наблюдений в обучающий датафрейм train
train = data.sample(frac=0.7, random_state=200)

# в исходном датафрейме data оставляем только те
# наблюдения, индексные метки которых отличаются
# от индекса наблюдений, попавших в обучающий датафрейм
# train, и записываем контрольный датафрейм test
test = data.drop(train.index)

#### Импутация пропусков
Выделяют 3 типа возникновения пропусков: MCAR, MAR, MNAR.

**MCAR** («совершенно случайно пропущенные» – Missing Completely At Random) –
тип возникнования пропусков, при котором вероятность пропуска для каждого
наблюдения набора одинакова. Вероятность пропуска значения для переменной
X не связана ни со значением самой переменной X, ни со значениями других переменных в наборе данных. Например, переменная Доход подчиняется условию
MCAR, если респонденты, которые не сообщают о своем доходе, имеют в среднем
такой же размер дохода, что и респонденты, которые указывают свой доход.

**MAR** («случайно пропущенные» – Missing At Random) — тип возникновения
пропусков, когда данные пропущены не случайно, а ввиду некоторых закономерностей. Вероятность пропуска значения для переменной X может быть объяснена другими имеющимися переменными, не содержащими пропусков. Например,
переменная Доход подчиняется условию MAR, если вероятность пропуска данных
в переменной Доход зависит от наблюдаемой переменной, например от переменной Образование. Например, респонденты с низким уровнем образования могут
иметь большее количество пропущенных значений дохода. Необходимо проанализировать взаимосвязь между переменной Доход и переменной Образование.

**MNAR** («не случайно пропущенные» – Missing Not At Random) — тип пропущенных даных, когда пропуск значения не является совершенно случайным и не
может быть полностью объяснен другими переменными в наборе. Пропущенные
значения остаются зависимыми от неизвестных нам факторов, необходимо провести дополнительные исследования.


In [33]:
# выводим информацию о количестве пропусков
# по каждой переменной в обучающей выборке
train.isna().sum()

gender                     0
age                        1
marital_status             1
job_position               0
credit_sum                 1
credit_month               0
tariff_id                  0
score_shk                  4
education                  2
living_region              0
monthly_income             2
credit_count            6477
overdue_credit_count    6477
open_account_flg           0
tariff                     0
dtype: int64

In [34]:
# выводим информацию о количестве пропусков по каждой переменной в контрольной выборке
test.isna().sum()

gender                     0
age                        2
marital_status             2
job_position               0
credit_sum                 1
credit_month               0
tariff_id                  0
score_shk                  3
education                  3
living_region              0
monthly_income             3
credit_count            2753
overdue_credit_count    2753
open_account_flg           0
tariff                     0
dtype: int64

**Обратите внимание, что импутацию средним, медианой и прочими статистиками необходимо выполнять ПОСЛЕ разбиения набора данных на обучающую и контрольную выборки.**

Если выполнить импутацию на всем наборе, а потом
разбить его на обучающую и контрольную выборки, получится, что при вычислении статистик для импутации использовались все наблюдения набора, часть из
которых потом у нас вошла в контрольную выборку (по сути, выборку новых данных). Поэтому получается, что статистики для импутации, которые мы получили
на всем наборе, пришли к нам частично из «будущего» (из новой, контрольной
выборки, которой по факту еще нет). 

Давайте заменим пропуски по количественной переменной age в обучающей
выборке медианой. Для этого необходимо воспользоваться методом .fillna(), задав с помощью параметра inplace значение True, чтобы выполнить импутацию на
месте. Пропуски по количественной переменной age в **контрольной выборке** заменяем медианой, вычисленной на обучающей выборке.

In [35]:
# заполняем пропуски в переменной age медианой
train_age_medium = train['age'].median()

train['age'] = train['age'].fillna(train_age_medium)
test['age'] = test['age'].fillna(train_age_medium)

In [36]:
# заполняем пропуски в переменных credit_sum, score_shk и monthly_income медианами
for i in ['credit_sum', 'score_shk', 'monthly_income']:
    train[i] = train[i].fillna(train[i].median())
    test[i] = test[i].fillna(train[i].median())

**Вновь обратите внимание, что пропуски в обучающей и контрольной выборках
импутированы медианами, вычисленными на обучающей выборке.**

In [37]:
# заполняем пропуски в переменных credit_count, overdue_credit_count -1
for i in ['credit_count', 'overdue_credit_count']:
    train[i] = train[i].fillna(-1)
    test[i] = test[i].fillna(-1)
   

Теперь приступаем к импутации пропусков в категориальных переменных.
Выведем частоты категорий переменной **marital_status** в обучающей и контрольной
выборках c помощью метода .value_counts().

In [38]:
# выводим частоты категорий для переменной marital_status
print(train['marital_status'].value_counts(dropna=False))
print(test['marital_status'].value_counts(dropna=False))

marital_status
MAR    65652
UNM    36581
DIV    11918
CIV     2942
WID     2428
NaN        1
Name: count, dtype: int64
marital_status
MAR    28302
UNM    15568
DIV     5051
CIV     1254
WID     1047
NaN        2
Name: count, dtype: int64


Заменим пропуски модой – самой часто встречающейся категорией, в данном
случае категорией MAR. Моду мы еще можем вычислить с помощью метода .mode().

In [39]:
# вычисляем моду для переменной education
train['marital_status'].mode()

0    MAR
Name: marital_status, dtype: object

In [40]:
# выполняем импутацию пропусков модой
train['marital_status'] = train['marital_status'].fillna('MAR')
test['marital_status'] = test['marital_status'].fillna('MAR')

In [41]:
# выводим частоты категорий для переменной marital_status
print(train['marital_status'].value_counts(dropna=False))
print(test['marital_status'].value_counts(dropna=False))

marital_status
MAR    65653
UNM    36581
DIV    11918
CIV     2942
WID     2428
Name: count, dtype: int64
marital_status
MAR    28304
UNM    15568
DIV     5051
CIV     1254
WID     1047
Name: count, dtype: int64


In [42]:
# выводим частоты категорий для переменной education
print(train['education'].value_counts(dropna=False))
print(train['education'].value_counts(dropna=False))

education
SCH    61126
GRD    50928
UGR     6973
PGR      418
ACD       75
NaN        2
Name: count, dtype: int64
education
SCH    61126
GRD    50928
UGR     6973
PGR      418
ACD       75
NaN        2
Name: count, dtype: int64


In [43]:
edu_mode = train['education'].mode()
train['education'] = train['education'].fillna(edu_mode)
test['education'] = test['education'].fillna(edu_mode)

#### Конструирование новых признаков
Конструирование новых признаков – неотъемлемый этап предварительной подготовки данных.

Выделяют два типа конструирования признаков – статическое
конструирование признаков (static feature engineering) и динамическое конструирование признаков (dynamic feature engineering).

**Статическое конструирование** признаков подразумевает, что мы создаем признаки вручную. Мы создаем фактические признаки (они присутствуют в нашем наборе данных) и подаем на вход
модели.

**Динамическое конструирование** признаков подразумевает, что мы создаем признаки в ходе построения модели «на лету», эти признаки создаются только в процессе обучения модели и в наш набор данных не записываются.

Каждый тип можно разделить еще на два подтипа: конструирование признаков, исходя из предметной области, опыта, бизнес-логики (domain-based feature engineering),
и конструирование признаков, исходя из особенностей алгоритма (algorithm-
based feature engineering). Мы начнем со статического конструирования признаков, исходя из предметной области, опыта, бизнес-логики. Чаще всего в рамках
такого способа конструирования признаков создают агрегаты.

**Агрегатная переменная** – переменная, полученная в результате математических операций над
двумя и более независимыми переменными.

In [44]:
# создаем переменную paym, которая является отношением выданной суммы кредита
# (credit_sum) к сроку кредита (credit_month), то есть ежемесячной суммой кредита
train['paym'] = train['credit_sum'] / train['credit_month']
test['paym'] = test['credit_sum'] / test['credit_month']

Кроме того, создадим переменную pti, которая является отношением ежемесячной суммы кредита к ежемесячному заработку. При вычислении переменных, которые являются отношениями количественных признаков, в случае когда происходит деление на ноль, могут быть получены бесконечные значения (значения **Inf и –Inf**). Их нужно заменить на конкретное значение. В данном случае заменим их на 1.

In [45]:
# создаем переменную pti, которая является отношением ежемесячной суммы кредита
# (paym) к ежемесячному заработку (monthly_income)
train['pti'] = train['paym'] / train['monthly_income']
test['pti'] = test['paym'] / test['monthly_income']

# заменяем бесконечные значения на 1
train['pti'] = train['pti'].replace([np.inf, -np.inf], 1)
test['pti'] = test['pti'].replace([np.inf, -np.inf], 1)

#### Создание переменной, у которой значения основаны на значениях исходной переменной

Часто при конструировании признаков, исходя из предметной области, нужно
проявить фантазию, и вы должны понимать, что сам признак – «шкатулка с секретом», часто просто указатель, в какую сторону нужно двигаться, чтобы извлечь
ценную информацию. 

Например, нередко приходится работать с таким предиктором, как сфера занятости, который обычно состоит из множества категорий.
Сама по себе сфера занятости может быть слабо связана с кредитоспособностью,
однако если мы заменим сферу занятости средней заработной платой в этой сфере, количеством банкротств предприятий в соответствующей сфере, количеством месяцев, в течение которого уволенный сотрудник, занятый в этой сфере, находит работу, мы можем получить более полезный признак. 

Регионы мы можем заменить средней заработной платой в регионе, размером и глубиной просрочки
в регионе, криминогенностью региона. Чаще всего мы подобные операции выполняем через создание переменной, у которой значения основаны на значениях исходной переменной.

In [46]:
# Выводим уникальные значения исходной переменной,
# в данном случае – значения переменной job_position
print(train['job_position'].unique())

['SPC' 'UMN' 'BIS' 'DIR' 'PNA' 'ATP' 'OTHER' 'WRK' 'BIU' 'NOR' 'WOI' 'INP'
 'PNI' 'WRP']


Допустим, мы получили из какого-то внешнего источника информацию о средней заработной плате клиентов с различной сферой занятости. 

Тогда создаем словарь, в котором ключом будет значение исходной переменной job_position, а значением – значение нашей будущей переменной avrzarplata.

In [47]:
# затем создаем словарь, в котором ключом будет значение
# исходной переменной job_position, а значением – значение
# будущей переменной avrzarplata
dct = {'UMN': 51000, 'SPC': 63000, 'INP': 55000, 'DIR': 60000,
'ATP': 46000, 'PNA': 71000, 'BIS': 86000, 'WOI': 76000,
'NOR': 54000,'WRK': 77000, 'WRP': 75000, 'PNV': 67000,
'BIU': 43000, 'PNI': 69000, 'HSK': 74000, 'PNS': 44000,
'INV': 88000, 'ONB': 62000, 'OTHER': 20000}

In [48]:
# создаем новую переменную avrzarplata, у которой значения
# сопоставлены значениям переменной job_position
train['avrzarplata'] = train['job_position'].map(dct)
train.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg,tariff,paym,pti,avrzarplata
53397,M,28.0,UNM,SPC,33579.0,10,1_4,0.431360,SCH,АМУРСКАЯ,36000.0,2.0,0.0,0,1.40,3357.900000,0.093275,63000
143962,M,38.0,UNM,SPC,23511.0,10,1_32,0.358472,GRD,СТАВРОПОЛЬСКИЙ,45000.0,2.0,0.0,0,1.32,2351.100000,0.052247,63000
146922,F,25.0,MAR,SPC,39990.0,24,1_5,0.613475,GRD,РОСТОВСКАЯ,50000.0,6.0,0.0,0,1.50,1666.250000,0.033325,63000
63697,F,65.0,UNM,UMN,3490.0,6,1_3,0.371355,GRD,ХАКАСИЯ,35000.0,4.0,0.0,0,1.30,581.666667,0.016619,51000
54503,M,49.0,MAR,BIS,36358.0,10,1_6,0.645187,SCH,ЧЕЛЯБИНСКАЯ,50000.0,2.0,0.0,0,1.60,3635.800000,0.072716,86000


In [49]:
# class MyDict(dict):
#     def __missing__(self, key):
#         return 'unknown'
# d = {'cat': 'kitten', 'dog': 'puppy'}
# st = pd.Series(['cat', 'dog', np.nan, 'rabbit'])
# res = st.map(MyDict(d))
# res

In [50]:
# удалим переменную avrzarplata
train.drop('avrzarplata', axis=1, inplace=True)

#### Создание бинарной переменной на основе значений количественных переменных

In [51]:
# создаем новую переменную retired, которая принимает
# значение "Yes", если значение переменной age больше 60,
# и значение "No" в противном случае
train['retired'] = np.where(train['age']>=60, 'Yes', 'No')
train.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg,tariff,paym,pti,retired
53397,M,28.0,UNM,SPC,33579.0,10,1_4,0.431360,SCH,АМУРСКАЯ,36000.0,2.0,0.0,0,1.40,3357.900000,0.093275,No
143962,M,38.0,UNM,SPC,23511.0,10,1_32,0.358472,GRD,СТАВРОПОЛЬСКИЙ,45000.0,2.0,0.0,0,1.32,2351.100000,0.052247,No
146922,F,25.0,MAR,SPC,39990.0,24,1_5,0.613475,GRD,РОСТОВСКАЯ,50000.0,6.0,0.0,0,1.50,1666.250000,0.033325,No
63697,F,65.0,UNM,UMN,3490.0,6,1_3,0.371355,GRD,ХАКАСИЯ,35000.0,4.0,0.0,0,1.30,581.666667,0.016619,Yes
54503,M,49.0,MAR,BIS,36358.0,10,1_6,0.645187,SCH,ЧЕЛЯБИНСКАЯ,50000.0,2.0,0.0,0,1.60,3635.800000,0.072716,No


In [52]:
# создаем новую переменную age_inc, которая принимает значение "Yes",
# если речь идет о клиентах старше 35 лет и с суммой кредита свыше 10 000,
# и значение "No" в противном случае
train['age_inc'] = np.where((train['age'] > 35) & (train['credit_sum'] > 10000), 'Yes', 'No')
train.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg,tariff,paym,pti,retired,age_inc
53397,M,28.0,UNM,SPC,33579.0,10,1_4,0.431360,SCH,АМУРСКАЯ,36000.0,2.0,0.0,0,1.40,3357.900000,0.093275,No,No
143962,M,38.0,UNM,SPC,23511.0,10,1_32,0.358472,GRD,СТАВРОПОЛЬСКИЙ,45000.0,2.0,0.0,0,1.32,2351.100000,0.052247,No,Yes
146922,F,25.0,MAR,SPC,39990.0,24,1_5,0.613475,GRD,РОСТОВСКАЯ,50000.0,6.0,0.0,0,1.50,1666.250000,0.033325,No,No
63697,F,65.0,UNM,UMN,3490.0,6,1_3,0.371355,GRD,ХАКАСИЯ,35000.0,4.0,0.0,0,1.30,581.666667,0.016619,Yes,No
54503,M,49.0,MAR,BIS,36358.0,10,1_6,0.645187,SCH,ЧЕЛЯБИНСКАЯ,50000.0,2.0,0.0,0,1.60,3635.800000,0.072716,No,Yes


In [53]:
# удаляем переменные retired и age_inc
train.drop(['retired', 'age_inc'], axis='columns', inplace=True)

#### Создание переменной, у которой каждое значение – среднее значение количественной переменной, взятое по уровню категориальной переменной

In [54]:
# пишем функцию, создающую переменную, у которой каждое значение будет средним значением
# количественной переменной (real_feature), взятым по уровню категориальной
# переменной (cat_feature)
def code_mean(df, cat_feature, real_feature):
    return (df[cat_feature].map(df.groupby(cat_feature)[real_feature].mean()))

In [55]:
# cоздаем переменную, у которой каждое значение – среднее значение monthly_income
# в категории переменной living_region
train['region_mean_income'] = code_mean(train, 'living_region', 'monthly_income')
train.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg,tariff,paym,pti,region_mean_income
53397,M,28.0,UNM,SPC,33579.0,10,1_4,0.431360,SCH,АМУРСКАЯ,36000.0,2.0,0.0,0,1.40,3357.900000,0.093275,38026.071197
143962,M,38.0,UNM,SPC,23511.0,10,1_32,0.358472,GRD,СТАВРОПОЛЬСКИЙ,45000.0,2.0,0.0,0,1.32,2351.100000,0.052247,32589.184487
146922,F,25.0,MAR,SPC,39990.0,24,1_5,0.613475,GRD,РОСТОВСКАЯ,50000.0,6.0,0.0,0,1.50,1666.250000,0.033325,34645.748347
63697,F,65.0,UNM,UMN,3490.0,6,1_3,0.371355,GRD,ХАКАСИЯ,35000.0,4.0,0.0,0,1.30,581.666667,0.016619,33443.197674
54503,M,49.0,MAR,BIS,36358.0,10,1_6,0.645187,SCH,ЧЕЛЯБИНСКАЯ,50000.0,2.0,0.0,0,1.60,3635.800000,0.072716,34010.612707


In [56]:
# удалим переменную region_mean_income
train.drop('region_mean_income', axis='columns', inplace=True)

#### Возведение в квадрат
Часто новые переменные создают просто за счет возведения исходных количественных переменных в степень. Например, создадим новые переменные, возведя
значения количественных переменных tariff, age, credit_sum, score_shk, monthly_in-
come, credit_month и credit_count в квадрат. Из-за возведения в квадрат у нас могут
получиться очень большие значения, а для представления очень больших значений в некоторых переменных используется экспоненциальная запись. Поэтому отключим ее.

In [57]:
# отключаем экспоненциальную запись
pd.set_option('display.float_format', lambda x: '%.3f' % x)
train.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg,tariff,paym,pti
53397,M,28.000,UNM,SPC,33579.000,10,1_4,0.431,SCH,АМУРСКАЯ,36000.000,2.000,0.000,0,1.400,3357.900,0.093
143962,M,38.000,UNM,SPC,23511.000,10,1_32,0.358,GRD,СТАВРОПОЛЬСКИЙ,45000.000,2.000,0.000,0,1.320,2351.100,0.052
146922,F,25.000,MAR,SPC,39990.000,24,1_5,0.613,GRD,РОСТОВСКАЯ,50000.000,6.000,0.000,0,1.500,1666.250,0.033
63697,F,65.000,UNM,UMN,3490.000,6,1_3,0.371,GRD,ХАКАСИЯ,35000.000,4.000,0.000,0,1.300,581.667,0.017
54503,M,49.000,MAR,BIS,36358.000,10,1_6,0.645,SCH,ЧЕЛЯБИНСКАЯ,50000.000,2.000,0.000,0,1.600,3635.800,0.073


In [58]:
# создаем новые переменные, возведя некоторые количественные переменные в квадрат
train['tariff_sq'] = train['tariff']**2
test['tariff_sq'] = test['tariff']**2
train['age_sq'] = train['age']**2
test['age_sq'] = test['age']**2
train['credit_sum_sq'] = train['credit_sum']**2
test['credit_sum_sq'] = test['credit_sum']**2
train['score_sq'] = train['score_shk']**2
test['score_sq'] = test['score_shk']**2
train['income_sq'] = train['monthly_income']**2
test['income_sq'] = test['monthly_income']**2
train['credit_month_sq'] = train['credit_month']**2
test['credit_month_sq'] = test['credit_month']**2
train['credit_count_sq'] = train['credit_count']**2
test['credit_count_sq'] = test['credit_count']**2

#### Дамми-кодирование (One-hot Encoding)
Библиотека pandas предлагает очень простой способ прямого кодирования с помощью функции **pd.get_dummies()**. Параметр drop_first задает тип дамми-кодирования. По умолчанию для этого параметра задано значение False и выполняется
дамми-кодирование по методу неполного ранга, в противном случае будет выполнено дамми-кодирование по методу полного ранга. Функция pd.get_dummies()
автоматически преобразует заданную категориальную переменную в набор дамми-переменных. Давайте преобразуем переменную marital_status в набор даммипеременных по методу неполного ранга.

In [59]:
# выполняем дамми-кодирование переменной marital_status по методу неполного ранга
dummies_unfull_rank_marital_status = pd.get_dummies(train['marital_status'], dtype='float')

# выводим первые 5 наблюдений
dummies_unfull_rank_marital_status.head()

,CIV,DIV,MAR,UNM,WID
53397,0.000,0.000,0.000,1.000,0.000
143962,0.000,0.000,0.000,1.000,0.000
146922,0.000,0.000,1.000,0.000,0.000
63697,0.000,0.000,0.000,1.000,0.000
54503,0.000,0.000,1.000,0.000,0.000


In [60]:
# выполняем дамми-кодирование переменной marital_status по методу полного ранга
dummies_unfull_rank_marital_status = pd.get_dummies(train['marital_status'], drop_first=True, dtype='float')

# выводим первые 5 наблюдений
dummies_unfull_rank_marital_status.head()

,DIV,MAR,UNM,WID
53397,0.000,0.000,1.000,0.000
143962,0.000,0.000,1.000,0.000
146922,0.000,1.000,0.000,0.000
63697,0.000,0.000,1.000,0.000
54503,0.000,1.000,0.000,0.000


#### Кодирование контрастами (Effect Coding)

Кодирование контрастами похоже на дамми-кодирование по методу полного
ранга с той только разницей, что в опорном уровне, представленном нулями, нули
заменяются на –1.

In [61]:
# выполняем кодирование контрастами
effects_marital_status = pd.get_dummies(train['marital_status'], dtype='float')

# убираем последний столбец наших dummy переменных
effects_marital_status = effects_marital_status.iloc[:,:-1]

# если во всей строку одни нули, то заменяем их на -1
effects_marital_status.loc[np.all(effects_marital_status == 0, axis=1)] = -1

# выводим первые 10 наблюдений
effects_marital_status.head(10)

,CIV,DIV,MAR,UNM
53397,0.000,0.000,0.000,1.000
143962,0.000,0.000,0.000,1.000
146922,0.000,0.000,1.000,0.000
63697,0.000,0.000,0.000,1.000
54503,0.000,0.000,1.000,0.000
115316,0.000,0.000,0.000,1.000
16354,0.000,0.000,0.000,1.000
98227,-1.000,-1.000,-1.000,-1.000
69607,0.000,0.000,1.000,0.000
99745,0.000,0.000,0.000,1.000


#### Присвоение категориям в лексикографическом порядке целочисленных значений, начиная с 0 (Label Encoding)

Категориям переменной в лексикографическом порядке можно присвоить целочисленные значения (начиная с 0), и в итоге категориальная переменная теряет свою категориальную природу и превращается в количественную.

Допустим, у нас есть переменная с 4 категориями A, B, C и D. Тогда мы присвоим
A – 0, B – 1, C – 2, D – 3. Эту процедуру можно выполнить с помощью класса LabelEncoder.

In [62]:
# импортируем класс LabelEncoder
from sklearn.preprocessing import LabelEncoder

# создаем экземпляр класса LabelEncoder
label_encoder = LabelEncoder().fit(train['job_position'])

# выполняем кодировку
train['job_position2'] = label_encoder.transform(train['job_position'])
train.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,...,paym,pti,tariff_sq,age_sq,credit_sum_sq,score_sq,income_sq,credit_month_sq,credit_count_sq,job_position2
53397,M,28.000,UNM,SPC,33579.000,10,1_4,0.431,SCH,АМУРСКАЯ,...,3357.900,0.093,1.960,784.000,1127549241.000,0.186,1296000000.000,100,4.000,9
143962,M,38.000,UNM,SPC,23511.000,10,1_32,0.358,GRD,СТАВРОПОЛЬСКИЙ,...,2351.100,0.052,1.742,1444.000,552767121.000,0.129,2025000000.000,100,4.000,9
146922,F,25.000,MAR,SPC,39990.000,24,1_5,0.613,GRD,РОСТОВСКАЯ,...,1666.250,0.033,2.250,625.000,1599200100.000,0.376,2500000000.000,576,36.000,9
63697,F,65.000,UNM,UMN,3490.000,6,1_3,0.371,GRD,ХАКАСИЯ,...,581.667,0.017,1.690,4225.000,12180100.000,0.138,1225000000.000,36,16.000,10
54503,M,49.000,MAR,BIS,36358.000,10,1_6,0.645,SCH,ЧЕЛЯБИНСКАЯ,...,3635.800,0.073,2.560,2401.000,1321904164.000,0.416,2500000000.000,100,4.000,1


**Мы можем выполнить Label Encoding как до разбиения на обучающую и контрольную выборки, так и после разбиения, поскольку не делаем никаких вычислений (разумеется, лучше выполнить Label Encoding до разбиения, чтобы не писать лишний программный код).**

На практике Label Encoding делают вручную и чаще всего для порядковых переменных. Для номинальных переменных кодировка Label Encoding редко бывает эффективна.

Например, у нас есть переменная Цвет автомобиля с категориями Синий, Желтый, Красный, мы можем закодировать их как 2, 1 и 0, но это не будет иметь большого смысла, ведь, выполнив такую кодировку, мы предполагаем определенный порядок 0 < 1 < 2, который здесь не выполняется.

In [63]:
# удалим переменную job_position2
train.drop('job_position2', axis=1, inplace=True)

#### Создание переменной, у которой каждое значение – частота наблюдений в категории переменных (Frequency Encoding)

Категориям переменной присваиваем абсолютные или относительные частоты.

In [64]:
# cоздаем переменную region_abs_freq, у которой каждое значение – абсолютная
# частота наблюдений в категории переменной living_region
abs_freq = train['living_region'].value_counts()
train['region_abs_freq'] = train['living_region'].map(abs_freq)
test['region_abs_freq'] = test['living_region'].map(abs_freq)
train.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,...,paym,pti,tariff_sq,age_sq,credit_sum_sq,score_sq,income_sq,credit_month_sq,credit_count_sq,region_abs_freq
53397,M,28.000,UNM,SPC,33579.000,10,1_4,0.431,SCH,АМУРСКАЯ,...,3357.900,0.093,1.960,784.000,1127549241.000,0.186,1296000000.000,100,4.000,618
143962,M,38.000,UNM,SPC,23511.000,10,1_32,0.358,GRD,СТАВРОПОЛЬСКИЙ,...,2351.100,0.052,1.742,1444.000,552767121.000,0.129,2025000000.000,100,4.000,3236
146922,F,25.000,MAR,SPC,39990.000,24,1_5,0.613,GRD,РОСТОВСКАЯ,...,1666.250,0.033,2.250,625.000,1599200100.000,0.376,2500000000.000,576,36.000,3024
63697,F,65.000,UNM,UMN,3490.000,6,1_3,0.371,GRD,ХАКАСИЯ,...,581.667,0.017,1.690,4225.000,12180100.000,0.138,1225000000.000,36,16.000,344
54503,M,49.000,MAR,BIS,36358.000,10,1_6,0.645,SCH,ЧЕЛЯБИНСКАЯ,...,3635.800,0.073,2.560,2401.000,1321904164.000,0.416,2500000000.000,100,4.000,3620


In [65]:
# cоздаем переменную region_rel_freq, у которой каждое значение – относительная
# частота наблюдений в категории переменной living_region
rel_freq = train['living_region'].value_counts() / len(train['living_region'])
train['region_rel_freq'] = train['living_region'].map(rel_freq)
test['region_rel_freq'] = test['living_region'].map(rel_freq)
train.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,...,pti,tariff_sq,age_sq,credit_sum_sq,score_sq,income_sq,credit_month_sq,credit_count_sq,region_abs_freq,region_rel_freq
53397,M,28.000,UNM,SPC,33579.000,10,1_4,0.431,SCH,АМУРСКАЯ,...,0.093,1.960,784.000,1127549241.000,0.186,1296000000.000,100,4.000,618,0.005
143962,M,38.000,UNM,SPC,23511.000,10,1_32,0.358,GRD,СТАВРОПОЛЬСКИЙ,...,0.052,1.742,1444.000,552767121.000,0.129,2025000000.000,100,4.000,3236,0.027
146922,F,25.000,MAR,SPC,39990.000,24,1_5,0.613,GRD,РОСТОВСКАЯ,...,0.033,2.250,625.000,1599200100.000,0.376,2500000000.000,576,36.000,3024,0.025
63697,F,65.000,UNM,UMN,3490.000,6,1_3,0.371,GRD,ХАКАСИЯ,...,0.017,1.690,4225.000,12180100.000,0.138,1225000000.000,36,16.000,344,0.003
54503,M,49.000,MAR,BIS,36358.000,10,1_6,0.645,SCH,ЧЕЛЯБИНСКАЯ,...,0.073,2.560,2401.000,1321904164.000,0.416,2500000000.000,100,4.000,3620,0.030


**Обратите внимание, что поскольку мы используем вычисления, то кодировку
частотами нужно выполнять строго после разбиения на обучающую и контрольную выборки.**

Частотами, вычисленными для категорий переменной в обучающей
выборке, заменяем категории переменной в обучающей и контрольной выборках. 

**Следует помнить, что в обучающем наборе могут встречаться не все возможные категории предиктора, некоторые категории могут присутствовать только в тестовом наборе или появятся только тогда, когда модель будет применяться к новым данным.** Поэтому схема кодировки должна предусмотреть возможность обработки новых категорий.

**На практике чаще используют кодирование относительными частотами, потому что абсолютные частоты сильно зависят от размера набора данных. Кодирование частотами может приводить к коллизиям, когда разные категории встречаются одинаковое количество раз и мы должны присвоить им одинаковые частоты.**

В таких случаях можно попробовать добавить случайный шум, добавить константу или
объединить, если исходные категории не отличаются по зависимой переменной (можно использовать тест хи-квадрат), и посмотреть, какой вариант дает наилучший результат.

Часто бывает полезным выполнить логарифмическое преобразование полученной переменной.

In [66]:
# удалим переменные region_abs_freq и region_rel_freq
train.drop(['region_abs_freq', 'region_rel_freq'], axis=1, inplace=True)
test.drop(['region_abs_freq', 'region_rel_freq'], axis=1, inplace=True)

#### Кодирование вероятностями зависимой переменной (Likelihood Encoding)
Участники соревнований Kaggle часто используют кодирование вероятностями
зависимой переменной (Likelihood Encoding). Существует множество способов
такой кодировки.

**Например**, у нас есть предиктор Class. Он имеет категории A, B и C. Категория A
встречается в 4 наблюдениях. В 3 из 4 наблюдений зависимая переменная принимает значение 1. 3 делим на 4, получаем 0,75. Вероятность для категории A будет равна 0,75. По сути, мы заменяем категорию предиктора средним значением зависимой переменной в этой категории, отсюда второе название данной кодировки – кодирование обычным средним значением зависимой переменной (Mean
Target Encoding).

#### Кодировка средним значением зависимой переменной, сглаженным через сигмоидальную функцию
теория - стр. 412 - 414

In [67]:
# импортируем класс StratifiedKFold
from sklearn.model_selection import StratifiedKFold

# создаем экземпляр класса StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# создаем список из двух признаков и зависимой переменной
cat_cols = ['living_region', 'job_position', 'open_account_flg']

# создаем обучающий массив со значениями зависимой переменной
y_train = train.loc[:, 'open_account_flg'].astype('int')

# создаем обучающий массив признаков
X_train = train[cat_cols].drop('open_account_flg', axis='columns')

# создаем контрольный массив признаков
X_valid = test[cat_cols].drop('open_account_flg', axis='columns')

In [70]:
# пишем функцию, выполняющую кодирование средними значениями зависимой переменной
def maen_target_enc(X_train, y_train, X_valid, skf):
    # отключаем предупреждения Anaconda
    import warnings
    warnings.filterwarnings('ignore')
    
    # вычисляем глобальное среднее – среднее значение зависимой переменной в обучающем наборе
    glob_mean = y_train.mean()
    
    # конкатенируем обучающий массив с признаками (задается первым аргументом) и обучающий
    # массив с метками зависимой переменной (задается вторым аргументом) по оси столбцов
    X_train = pd.concat([X_train, pd.Series(y_train, name='open_account_flg')], axis='columns')

    # создаем копию массива признаков, получившегося в результате конкатенации
    new_X_train = X_train.copy()
    
    # создаем список с именами категориальных признаков,
    # который мы будем использовать ниже в циклах for
    cat_features = X_train.columns[X_train.dtypes == 'object'].tolist()
    
    # для каждого категориального признака создаем столбец, каждое
    # значение которого – глобальное среднее
    for col in cat_features:
        new_X_train[col + '_mean_target'] = [glob_mean for _ in range(new_X_train.shape[0])]

    # вычисляем среднее значение зависимой переменной в категории признака
    # по каждому блоку перекрестной проверки, используя данные вне этого блока
    # например, мы используем 5-блочную перекрестную проверку и нам нужно
    # вычислить среднее значение зависимой переменной для категории A в блоке 0,
    # для вычисления этого среднего значения используются лишь наблюдения в категории А
    # в обучающих блоках 1, 2, 3, 4, если вместо категорий у нас
    # значения NaN, заменяем глобальным средним, в итоге получаем новый обучающий набор
    for train_idx, valid_idx in skf.split(X_train, y_train):
        X_train_cv, X_valid_cv = X_train.iloc[train_idx, :], X_train.iloc[valid_idx, :]

        for col in cat_features:
            means = X_valid_cv[col].map(X_train_cv.groupby(col)['open_account_flg'].mean())
            X_valid_cv[col + '_mean_target'] = means.fillna(glob_mean)
            print(len(valid_idx), valid_idx.size)
            print(len(X_valid_cv), X_valid_cv.size)
            new_X_train.iloc[valid_idx] = X_valid_cv

    # удаляем из нового обучающего набора категориальные признаки и зависимую переменную
    new_X_train.drop(cat_features + ['open_account_flg'], axis='columns', inplace=True)

    # создаем копию контрольного массива признаков
    new_X_valid = X_valid.copy()

    # каждую категорию категориального признака в контрольном наборе
    # заменяем средним значением зависимой переменной в этой же категории
    # признака, вычисленным на обучающем наборе, значения NaN заменяем глобальным средним
    for col in cat_features:
        means = new_X_valid[col].map(X_train.gropby(col)['open_account_flg'].mean())
        new_X_valid[col + '_mean_target'] = means.fillna(glob_mean)
        
    # удаляем из контрольного набора категориальные признаки
    new_X_valid.drop(X_train.columns[X_train.dtypes == 'object'], axis='columnns', inplace=True)
    
    # возвращаем новые датафреймы
    return new_X_train, new_X_valid

In [72]:
# выполняем кодирование средними значениями зависимой переменной для переменных
# living_region и job_position в обучающем и контрольном наборах

train_mean_targer, valid_mean_target = maen_target_enc(X_train, y_train, X_valid, skf)

23905 23905
23905 95620


IndexError: single positional indexer is out-of-bounds